In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
from sqlalchemy import create_engine, VARCHAR
from sqlalchemy import Table, MetaData, Column
from tqdm import tqdm

### Identifiants

In [ ]:
# Informations d'identification PostgreSQL
db_user = "terence"
db_password = "Canon0-Spinout8-Debatable6-Proofread2"
db_host = "stockage-donnee-foncieres-gr02.postgres.database.azure.com"
db_port = 5432
db_name = "postgres"

In [ ]:
# Créez une connexion à la base de données
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}')

In [ ]:
# Chargez le fichier CSV dans un DataFrame Pandas
df = pd.read_csv('./src/datasets/csv/valeurs_foncieres_affichage.csv', encoding='utf-8', dtype={'code_commune': str, 'code_departement': str})


In [ ]:
# Nom de la table dans la base de données
table_name = 'valeur_fonciere'

In [ ]:
# Supprimer la colonne inutile "Unnamed: 0"
df = df.drop(columns=['Unnamed: 0'])

In [ ]:
# Nombre total de lignes dans le DataFrame
total_rows = len(df)

# Barre de progression tqdm
with tqdm(total=total_rows, desc="Insertion des données dans la base de données") as pbar:
    # Commencez la transaction
    with engine.connect() as connection:
        with connection.begin():
            # Créez la table en fonction de l'en-tête du fichier CSV
            metadata = MetaData()
            table = Table(table_name, metadata, *[Column(col, VARCHAR) for col in df.columns])
            metadata.create_all(connection)

            # Itérez sur les lignes du DataFrame
            for index, row in df.iterrows():
                # Insérez la ligne dans la table
                connection.execute(table.insert().values(**row.to_dict()))

                # Incrémente la barre de progression
                pbar.update(1)

print("Les données ont été insérées avec succès.")